In [1]:
%%writefile configs/efficientnet/efficientnetb0_alzheimer_axial_view.py
_base_ = [
    # '../_base_/models/efficientnet_v2/efficientnetv2_m.py',
    '../_base_/models/efficientnet_b0.py',
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    # '../_base_/schedules/imagenet_bs256.py',
    '../_base_/default_runtime.py'
]

pretrained = 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth'

model = dict(
    type='ImageClassifier', 
    
    backbone=dict(
        type='EfficientNet',
        arch='b0',
        init_cfg = dict(
            type='Pretrained', 
            checkpoint=pretrained,
            prefix='backbone',
        )
    ),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        num_classes=3,
        topk = (1, 3),
        # cal_acc=True,
    ))

dataset_type = 'CustomDataset'
data_preprocessor = dict(
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  
    # dict(type='EfficientNetRandomCrop', scale=224, crop_padding=0),     # Random scaling and cropping
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='PackInputs'),         # prepare images and labels
]

test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  
    # dict(type='EfficientNetCenterCrop', crop_size=224,crop_padding=0),     # center crop
    dict(type='PackInputs'),                 # prepare images and labels
]

train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='YoriDataset_vgg/train',
        classes='data/classes.txt',
        ann_file='train_ann.txt',
        with_label=True,
        pipeline=train_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=4,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='YoriDataset_vgg/validation',
        classes='data/classes.txt',
        ann_file='val_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=4,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='YoriDataset_vgg/test',
        classes='data/classes.txt',
        ann_file='test_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, 3))

test_evaluator = val_evaluator 

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=128)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Overwriting configs/efficientnet/efficientnetb0_alzheimer_axial_view.py


In [2]:
!python ./tools/train.py \
    configs/efficientnet/efficientnetb0_alzheimer_axial_view.py \
    --work-dir work_dirs/alzheimer/axial/efficientnet \
    # --auto-scale-lr

configs/efficientnet/efficientnetb0_alzheimer_axial_view.py
12/01 23:16:30 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1070086653
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags

# Predict images

In [3]:
import mmcv
from mmpretrain.apis import init_model
from mmpretrain import ImageClassificationInferencer
# from mmcls.apis import inference_model, init_model, show_result_pyplot


config_file = 'configs/efficientnet/efficientnetb0_alzheimer_axial_view.py'
checkpoint_file = 'work_dirs/alzheimer/axial/efficientnet/epoch_100.pth'

# Specify the device, if you cannot use GPU, you can also use CPU 
# by specifying `device='cpu'`.
device = 'cuda:0'
# device = 'cpu'

# Build the model according to the config file and load the checkpoint.
model_axial = init_model(config_file, checkpoint_file, device=device)

Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/efficientnet/epoch_100.pth


In [35]:
# %matplotlib inline
import json
# from mmcls.core.visualization import imshow_infos
import os
filepath_axial = 'data/YoriDataset_vgg/train'

wrong_predictions = []
for i in os.listdir(filepath_axial):
    temp = os.path.join(filepath_axial,i)
    for j in os.listdir(temp):
        temp1 = os.path.join(temp,j)      
        img_array =  mmcv.imread(temp1)
        inferencer = ImageClassificationInferencer(model_axial)
        # result['true_class'] = temp1.split('/')[3]
        result = inferencer(temp1)[0]
        original_class = temp1.split('/')[3]
        if result['pred_class'] != original_class:
            wrong_predictions.append(temp1)
        # ImageClassificationInferencer.__call__(inputs=img_array)

      
      # show_result_pyplot(model_axial, temp1, result)
print('\n'.join(wrong_predictions))        

data/YoriDataset_vgg/train/NC/55x-134.png
data/YoriDataset_vgg/train/NC/8x-116.png
data/YoriDataset_vgg/train/NC/93x-102.png
data/YoriDataset_vgg/train/NC/94x-105.png
data/YoriDataset_vgg/train/NC/86x-131.png
data/YoriDataset_vgg/train/NC/96x-97.png
data/YoriDataset_vgg/train/NC/25x-104.png
data/YoriDataset_vgg/train/NC/96x-99.png
data/YoriDataset_vgg/train/NC/20x-119.png
data/YoriDataset_vgg/train/NC/94x-103.png
data/YoriDataset_vgg/train/NC/95x-98.png
data/YoriDataset_vgg/train/NC/56x-134.png
data/YoriDataset_vgg/train/NC/46x-136.png
data/YoriDataset_vgg/train/NC/44x-116.png
data/YoriDataset_vgg/train/NC/88x-106.png
data/YoriDataset_vgg/train/NC/96x-96.png
data/YoriDataset_vgg/train/MCI/76x-116.png
data/YoriDataset_vgg/train/MCI/73x-145.png
data/YoriDataset_vgg/train/MCI/13x-137.png
data/YoriDataset_vgg/train/MCI/58x-131.png
data/YoriDataset_vgg/train/MCI/40x-154.png
data/YoriDataset_vgg/train/MCI/24x-95.png
data/YoriDataset_vgg/train/MCI/49x-124.png
data/YoriDataset_vgg/train/MCI/7x

In [53]:
import os
import shutil

heatmap_folder = 'data/efficientnet_heat/train'

for i in os.listdir(filepath_axial):
    class_folder = os.path.join(filepath_axial, i)
    for j in os.listdir(class_folder):
        img_name = os.path.join(class_folder, j)
        # print(img_name)

        copy_image = heatmap_folder + '/' + i + '/' + j
        # print(copy_image)
        if {img_name}.issubset(wrong_predictions)==False:
            shutil.copy(img_name, copy_image)
        # break
    # break
print('Done')

Done


In [4]:
import os
import subprocess

train_AD = 'data/efficientnet_heat/train/AD'
train_MCI = 'data/efficientnet_heat/train/MCI'
train_NC = 'data/efficientnet_heat/train/NC'

trainheat_AD = 'data/efficientnet_heat/train/AD'
trainheat_MCI = 'data/efficientnet_heat/train/MCI'
trainheat_NC = 'data/efficientnet_heat/train/NC'

count = 0
for i in os.listdir(train_AD):
    count+=1
    temp = os.path.join(train_AD,i)
    out = trainheat_AD + '/heat_'+i
    print(out)  
    subprocess.run(["python", "tools/visualization/vis_cam.py", 
                temp,
                 "configs/efficientnet/efficientnetb0_alzheimer_axial_view.py",
                 "work_dirs/alzheimer/axial/efficientnet/epoch_100.pth",
                 # "--target-layers", "backbone.features.16",
                 "--method", "GradCAM",
                 "--save-path", out,
                 "--eigen-smooth", "--aug-smooth"
                 ], text=True)
    print(count)
    # break

data/efficientnet_heat/train/AD/heat_x-129 (6).png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/efficientnet/epoch_100.pth
49
Automatically choose the last norm layer "backbone.layers.6.bn" as the target layer.
1
data/efficientnet_heat/train/AD/heat_x-126.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/efficientnet/epoch_100.pth
49
Automatically choose the last norm layer "backbone.layers.6.bn" as the target layer.
2
data/efficientnet_heat/train/AD/heat_x-105 (23).png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/efficientnet/epoch_100.pth
49
Automatically choose the last norm layer "backbone.layers.6.bn" as the target layer.
3
data/efficientnet_heat/train/AD/heat_x-121 (7).png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/efficientnet/epoch_100.pth
49
Automatically choose the last norm layer "backbone.layers.6.bn" as the target layer.
4
data/efficientnet_heat/train/AD/heat_x-121 (3).

In [38]:
%%writefile configs/efficientnet/efficientnetb0_alzheimer_axial_view_heat.py
_base_ = [
    '../_base_/models/efficientnet_v2/efficientnetv2_m.py',
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    # '../_base_/schedules/imagenet_bs256.py',
    '../_base_/default_runtime.py'
]

pretrained = 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth'

model = dict(
    type='ImageClassifier', 
    
    backbone=dict(
        type='EfficientNet',
        arch='b0',
        init_cfg = dict(
            type='Pretrained', 
            checkpoint=pretrained,
            prefix='backbone',
        )
    ),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        num_classes=3,
        topk = (1, 3),
        # cal_acc=True,
    ))

dataset_type = 'CustomDataset'
data_preprocessor = dict(
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=True)

train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  
    dict(type='EfficientNetRandomCrop', scale=224, crop_padding=0),     # Random scaling and cropping
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='PackInputs'),         # prepare images and labels
]

test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  
    dict(type='EfficientNetCenterCrop', crop_size=224,crop_padding=0),     # center crop
    dict(type='PackInputs'),                 # prepare images and labels
]

train_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='efficientnet_heat/train',
        classes='data/classes.txt',
        with_label=True,
        pipeline=train_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='efficientnet_heat/validation',
        classes='data/classes.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='efficientnet_heat/test',
        classes='data/classes.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, 3))

test_evaluator = val_evaluator 

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=128)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Overwriting configs/efficientnet/efficientnetb0_alzheimer_axial_view_heat.py


In [39]:
!python ./tools/train.py \
    configs/efficientnet/efficientnetb0_alzheimer_axial_view_heat.py \
    --work-dir work_dirs/alzheimer/axial/efficientnet_heat \
    # --auto-scale-lr

configs/efficientnet/efficientnetb0_alzheimer_axial_view_heat.py
11/26 19:26:57 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 916778201
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture f